## API de LastFM 


In [1]:
import requests
import base64
import pandas as pd
import mysql.connector

# Credenciales de tu app
CLIENT_ID = 'a80cf890990b4f3e9585bb385be538d0'
CLIENT_SECRET = '6445dff7ddd44f49a4c98f3da475a2f4'
# URL para obtener el token
url = 'https://accounts.spotify.com/api/token'
# Codifica tus credenciales
credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
encoded_credentials = base64.b64encode(credentials.encode()).decode()
# Encabezados y datos
headers = {
    'Authorization': f'Basic {encoded_credentials}',
}
data = {
    'grant_type': 'client_credentials',
}
# Solicitar el token
response = requests.post(url, headers=headers, data=data)
if response.status_code == 200:
    token = response.json().get('access_token')
    print(f"Access Token: {token}")
else:
    print(f"Error: {response.status_code}")
    print(response.json())

Access Token: BQAVwLSzre6ZG8VOM5Z7aufCwXKiaTaBuYwLuU6DHnxGrVRycr9fLcp1X609duIbbWxUpHn15dYyw0Qepa8t_Jj2AAP08H1SYxP737iTrBxgARieKeg


In [2]:
#funcion obtener info de spotify
# Géneros y rango de años
generos = ["pop", "rock", "jazz", "electronic"]
años = range(2019, 2023)  # De 2019 a 2023

# Esta función va a hacer la consulta y extraer la información
def obtener_info_spotify(generos, años):
    access_token = token

    # Encabezados con el token de acceso
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    resultados = []

    for genero in generos:
        for año in años:
            print(f"Buscando {genero} del año {año}...")
            url = "https://api.spotify.com/v1/search"
            params = {
                "q": f"genre:{genero} year:{año}",  # Filtrar por género y año
                "type": "album",  # Puedes cambiarlo a 'track' para canciones
                "limit": 50  # Número de resultados
            }

            # Hacemos la solicitud a la API
            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                print(f"Resultados para {genero} del año {año}: {len(data.get('albums', {}).get('items', []))}")  # Imprime la cantidad de resultados obtenidos
                print(data.get('albums', {}).get('items', []))

                # Recorremos los resultados
                for item in data.get('albums', {}).get('items', []):
                    # Recorremos los artistas
                    for artist in item["artists"]: 
                        if(artist["name"] != "Various Artists" and artist["name"] !="Genres Masters of Music"): #Descarto en el condicional nombre de artistas erroneos
                            album_info = {
                                "artista": artist["name"],  # Nombre del artista
                                "genero": genero,  # Género
                                "tipo": item["type"],  # Tipo de resultado (puede ser "track" o "album")
                                "nombre": item["name"],  # Nombre del álbum
                                "año_lanzamiento": item["release_date"].split("-")[0],  # Año de lanzamiento
                                "id": item["id"],  # ID del álbum
                            }
                        resultados.append(album_info)

            else:
                print(f"Error en la solicitud: {response.status_code}")
                print(response.json())

    return resultados

In [3]:
#Obtener información de lastfm para un artista
def obtener_info_de_un_artista_por_nombre(nombre_artista):
    artista = {"nombre": "", "biografia": "", "oyentes": "", "reproducciones": "", "similares": []}
    url_base = "http://ws.audioscrobbler.com/2.0/"  # Construir la URL para la solicitud de información del artista
    api_key = "9de743ec4e44e6ebfa1dd9a04931600d"
    url_artista = url_base + "?method=artist.getinfo&artist=" + nombre_artista + "&api_key=" + api_key + "&format=json"
    
    # Hacer la solicitud HTTP a la API
    response = requests.get(url_artista)
    
    if response.status_code == 200: #Si es correcto la solicitud ejecuta lo siguiente:
        data = response.json()
        artista['nombre']= data['artist']['name']
        artista['biografia']= data['artist']['bio']['summary']
        artista['oyentes'] = data['artist']['stats']['listeners']
        artista['reproducciones']= data['artist']['stats']['playcount']

        for artista_similar in data['artist']['similar']['artist']:
            artista['similares'].append(artista_similar['name'])
        
        return artista

    else:
        print(f"Error en la solicitud:{response.status_code}")
        print(response.json())

In [4]:
def obtener_tabla_artistas_de_albums(albums):
    tabla_lastfm_artista = []

    for album in albums: 
        datos_artista = obtener_info_de_un_artista_por_nombre(album["artista"])
        artista_info = {
            "artista": datos_artista["nombre"],  # Nombre del artista
            "oyentes": datos_artista["oyentes"],
            "reproducciones": datos_artista["reproducciones"],
            "similares": datos_artista["similares"],
            "biografia": datos_artista["biografia"],
        }
        tabla_lastfm_artista.append(artista_info)
    return tabla_lastfm_artista

In [5]:
# Llamamos a la función para obtener los resultados de un album
tabla_spotify_album = obtener_info_spotify(generos, años)
tabla_lastfm_artista = obtener_tabla_artistas_de_albums(tabla_spotify_album)

Buscando pop del año 2019...
Resultados para pop del año 2019: 2
[{'album_type': 'album', 'total_tracks': 14, 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW', 'DM', 'FJ', 'GM', 'GE', 'GD', 'GW', 'GY', 'HT', 'JM', 'KI', 'LS', 'LR', 'MW', 'MV', 'ML', 'MH', 'FM', 'NA', 'NR', 'NE', 'PW', 'PG', 'WS', 'SM', 'ST', 'SN', 'SC', 'SL', 'SB', 'KN', 'LC', 'VC', 'SR', 'TL', 'TO', 'TT'

In [6]:
# Convertimos los resultados en un DataFrame de pandas
dfalbum = pd.DataFrame(tabla_spotify_album)
print(dfalbum)

                                 artista      genero   tipo  \
0                             André Popp         pop  album   
1                       Typical Minority         pop  album   
2                            Pattawiguys         pop  album   
3                     The Tower of Dudes        rock  album   
4                              Scenester        rock  album   
5                       Ibiza DJ Rockerz        rock  album   
6                 Crazy Party Music Guys        rock  album   
7                                  Genre        rock  album   
8                            Onda Alegre        jazz  album   
9           Instrumental Jazz Music Guys        jazz  album   
10                          Instrumental        jazz  album   
11   Relaxing Instrumental Jazz Ensemble        jazz  album   
12                        Alphagarrsongs        jazz  album   
13             Jazz Night Music Paradise        jazz  album   
14                    Jazz Instrumentals        jazz  a

In [7]:
# Convertimos los resultados en un DataFrame de pandas
dfartista = pd.DataFrame(tabla_lastfm_artista)
print(dfartista)

                                 artista oyentes reproducciones  \
0                             André Popp   11811          96177   
1                       Typical Minority       2              3   
2                            Pattawiguys      39            226   
3                     The Tower of Dudes    1009           4189   
4                              Scenester      82            224   
5                       Ibiza DJ Rockerz    5023          49581   
6                 Crazy Party Music Guys      68            282   
7                                  Genre    2374           7921   
8                            Onda Alegre      41            316   
9           Instrumental Jazz Music Guys     243           5470   
10                          Instrumental  159545        1520557   
11   Relaxing Instrumental Jazz Ensemble   27597         394695   
12                        Alphagarrsongs      22             42   
13             Jazz Night Music Paradise     742           558

2- API de last.fm. Extraer detalles como:

Biografía del Artista.

Popularidad y Estadísticas de Reproducción.

Artistas Similares.

In [11]:
df_test = dfartista.drop('similares', axis=1)
df_test.head()

,artista,oyentes,reproducciones,biografia
0,André Popp,11811,96177,Andre Popp contributed two of the greatest alb...
1,Typical Minority,2,3,"<a href=""https://www.last.fm/music/Typical+Mi..."
2,Pattawiguys,39,226,"<a href=""https://www.last.fm/music/Pattawiguy..."
3,The Tower of Dudes,1009,4189,THE TOWER OF DUDES is a country-folk-punk proj...
4,Scenester,82,224,"Punky rock band from Albuquerque, New Mexico, ..."


In [10]:
dfartista.head()

,artista,oyentes,reproducciones,similares,biografia
0,André Popp,11811,96177,"[John Buzon Trio, Dean Elliott & His Big Band,...",Andre Popp contributed two of the greatest alb...
1,Typical Minority,2,3,[],"<a href=""https://www.last.fm/music/Typical+Mi..."
2,Pattawiguys,39,226,"[00.01, soulfear, BellyTheBear, Youth Brush, S...","<a href=""https://www.last.fm/music/Pattawiguy..."
3,The Tower of Dudes,1009,4189,"[Buzz Carlton, Astro-Hot, Zapruder Sequence, T...",THE TOWER OF DUDES is a country-folk-punk proj...
4,Scenester,82,224,"[Romeo Goes To Hell, The Hollis Wake, The Dirt...","Punky rock band from Albuquerque, New Mexico, ..."


In [12]:
from sqlalchemy import create_engine

# Configurar conexión a la base de datos MySQL
host = "127.0.0.1"  # Host de la base de datos (usualmente 'localhost')
port = "3306"  # Puerto de la base de datos
user = "root"  # Usuario de la base de datos
password = "AlumnaAdalab"  # Contraseña de la base de datos
database = "MusicStream"  # Nombre de tu base de datos

# Crear la cadena de conexión
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')

# Pasar el DataFrame a la base de datos
df_test.to_sql('lastFm', con=engine, if_exists='replace', index=False)

-1